In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data


D:\Anaconda\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#tensorboard无效的话，可以尝试降低其版本或者更新tensorflow版本；
# 载入数据集
mnist = input_data.read_data_sets("C:\\Users\\Administrator\\Desktop\\MNIST_data",one_hot=True)

#每个批次的大小
batch_size = 100
# 计算一共有多少个批次  //整除
n_batch = mnist.train.num_examples // batch_size

#参数概要
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean',mean)#平均值
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
        tf.summary.scalar('stddev',stddev)#标准差
        tf.summary.scalar('max',tf.reduce_max(var))#最大值
        tf.summary.scalar('min',tf.reduce_min(var))#最小值
        # tf.summary.scalar('histogram',var)#直方图
            

# 定义命名空间
with tf.name_scope('input'):
    # 定义两个placeholder
    x = tf.placeholder(tf.float32, [None,784],name='x-input')
    y = tf.placeholder(tf.float32,[None,10],name='y-input')
    
with tf.name_scope('layer'):
    #创建一个简单的神经网络 输入层 784个神经元； 输出层10个标签
    with tf.name_scope('weight'):
        W = tf.Variable(tf.zeros([784,10]),name='W')
        variable_summaries(W)
    with tf.name_scope('biases'):
        b = tf.Variable(tf.zeros([10]),name='b')
        variable_summaries(b)
    with tf.name_scope('wx_plus_b'):
        wx_plus_b = tf.matmul(x,W)+b
    with tf.name_scope('softmax'):
         prediction = tf.nn.softmax(wx_plus_b)


# 定义二次代价函数
# loss = tf.reduce_mean(tf.square(y-prediction))
with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))
    #他只有一个值，我们就看其值即可；
    tf.summary.scalar('loss',loss)
with tf.name_scope('train'):
    # 使用梯度下降法
    train_step = tf.train.GradientDescentOptimizer(0.2).minimize(loss)

#初始化变量
init = tf.global_variables_initializer()
with tf.name_scope('accuracy'):
    # 定义求准确率的方法
    # tf.argmax比较参数  ；tf.argmax求标签最大的值，在那个位置，相当于标签
    # 结果存放在一个布尔型列表中
    with tf.name_scope('correct_prediction'):
        correct_prediction = tf.equal(tf.argmax(y,axis = 1),tf.argmax(prediction,axis = 1))# argmax返回最大值所在位置
    with tf.name_scope('accuracy'):
        # 求准确率
        # 转换格式tf.cast;然后求平均值即可得到准确率；
        accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
        tf.summary.scalar('accuracy',accuracy)
        
# 合并所有的summary
merged = tf.summary.merge_all()

with tf.Session() as sess:
    sess.run(init)
    writer = tf.summary.FileWriter('logs/',sess.graph)
    for epoch in range(51):
        for batch in range(n_batch):
            #图片数据，图片标签；
            batch_xs,batch_ys = mnist.train.next_batch(batch_size)
            summary,_=sess.run([merged,train_step],feed_dict={x:batch_xs,y:batch_ys})

        writer.add_summary(summary,epoch)    
        acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels})
        print("Iter"+str(epoch)+",Testing Accuracy:"+str(acc))

Extracting C:\Users\Administrator\Desktop\MNIST_data\train-images-idx3-ubyte.gz
Extracting C:\Users\Administrator\Desktop\MNIST_data\train-labels-idx1-ubyte.gz
Extracting C:\Users\Administrator\Desktop\MNIST_data\t10k-images-idx3-ubyte.gz
Extracting C:\Users\Administrator\Desktop\MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

Iter0,Testing Accuracy:0.8245
Iter1,Testing Accuracy:0.8938
Iter2,Testing Accuracy:0.9007
Iter3,Testing Accuracy:0.9064
Iter4,Testing Accuracy:0.9085
Iter5,Testing Accuracy:0.9104
Iter6,Testing Accuracy:0.9116
Iter7,Testing Accuracy:0.9138
Iter8,Testing Accuracy:0.9145
Iter9,Testing Accuracy:0.9162
Iter10,Testing Accuracy:0.9174
Iter11,Testing Accuracy:0.918
Iter12,Testing Accuracy:0.9196
Iter13,Testing Accuracy:0.9188
Iter14,Testing Accuracy:0.9195
Iter15,Testing Accuracy:0.92
Iter16,Tes